In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

from src.ingestion.report import Report
from src.llm_agents.kpi_simple_extraction.agent import KPISimpleExtractionAgent

from warnings import  simplefilter

simplefilter(action='ignore', category=Warning)

%load_ext autoreload
%autoreload 2

In [ ]:
reports = []
list_paths = os.listdir("data")
for path in tqdm(list_paths):
    if path.endswith('.json'):
        report = Report.from_json("data/"+path)
        report.get_kpis()

In [ ]:
df = pd.DataFrame(reports)
df = df.explode("questions").reset_index(drop=True)
df = df.assign(**df["questions"].apply(pd.Series))
df = df.assign(**df["response"].apply(pd.Series))
df = df.assign(**df["KPI"].apply(pd.Series))
df = df.explode("values")
df = df.query("not values.isna()").reset_index(drop=True)
df = df.assign(**df["values"].apply(pd.Series))
df.found = df.found.apply(lambda x: 1 if x == "True" or x else 0)
df.drop(columns=["questions", "response", "KPI", "values"], inplace=True)

In [ ]:
df_sorted = df.sort_values(by=["file", "source"]).reset_index(drop=True)

In [ ]:
df_sorted.to_json("data/kpi_extraction.json", orient="records", lines=True)

In [ ]:
df_sorted